<img src="../static/logopython.png" alt="Logo Python" style="width: 300px; display: inline"/>
<img src="../static/deimoslogo.png" alt="Logo Deimos" style="width: 300px; display: inline"/>

# Clase 5: Metaprogramación en Python

En la metaprogramación, escribimos <strong>código que trabaja con código</strong>. Algo para lo que Python 3 está <strong>especialmente bien preparado</strong>

Veamos a continuación para qué nos puede interesar escribir código que trabaja con código.

## ¿Qué es eso de la metaprogramación?

Como ya hemos dicho, la metaprogramación consiste en escribir código que va a trabajar con el código ya escrito, incluso consigo mismo.

Al ser Python un lenguaje dinámico, no podemos saber las interioridades de los componentes con los que estemos trabajando, salvo que los inspeccionemos en tiempo de ejecución. Por ejemplo

In [33]:
# Añadimos atributos a una instancia de una clase, una vez creada
class Empleado(object):
    "Clase para definir a un empleado"
    def __init__(self, nombre, email):
        self.nombre = nombre
        self.email = email
        
    def getNombre(self):
        return self.nombre
    

jorge = Empleado("Jorge", "jorge@mail.com")
jorge.guapo = "Por supuesto"

print("Es Jorge guapo: {}".format(jorge.guapo))

Es Jorge guapo: Por supuesto


En la manera más básica de metaprogramación, podemos manipular los atributos de una clase, mediante el trio de funciones [hasattr](https://docs.python.org/3/library/functions.html#hasattr), [getattr](https://docs.python.org/3/library/functions.html#getattr), [setattr](https://docs.python.org/3/library/functions.html#setattr)

In [20]:
# Probando hasattr, getattr, setattr
print(hasattr(jorge, 'guapo'))
print(getattr(jorge, 'guapo'))

print(hasattr(jorge, 'listo'))
setattr(jorge, 'listo', 'Más que las monas')
print(getattr(jorge, 'listo'))

True
Por supuesto
False
Más que las monas


Además de eso, tenemos más propiedades a nuestra disposición:

In [21]:
# Nombre de la clase
Empleado.__name__

'Empleado'

In [23]:
# Identificador de creador
print(isinstance(jorge, Empleado))

True


In [24]:
# Acceso a docstring
print(Empleado.__doc__)

help(jorge)
help(Empleado)

Clase para definir a un empleado
Help on Empleado in module __main__ object:

class Empleado(builtins.object)
 |  Clase para definir a un empleado
 |  
 |  Methods defined here:
 |  
 |  __init__(self, nombre, email)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)

Help on class Empleado in module __main__:

class Empleado(builtins.object)
 |  Clase para definir a un empleado
 |  
 |  Methods defined here:
 |  
 |  __init__(self, nombre, email)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 

El módulo [inspect](https://docs.python.org/3/library/inspect.html) también se puede usar para acceder a las interioridades de un objeto

In [29]:
import inspect

print(inspect.getdoc(Empleado))
print(inspect.getmembers(jorge))


Clase para definir a un empleado
[('__class__', <class '__main__.Empleado'>), ('__delattr__', <method-wrapper '__delattr__' of Empleado object at 0x7f758826cc88>), ('__dict__', {'listo': 'Más que las monas', 'nombre': 'Jorge', 'email': 'jorge@mail.com', 'guapo': 'Por supuesto'}), ('__dir__', <built-in method __dir__ of Empleado object at 0x7f758826cc88>), ('__doc__', 'Clase para definir a un empleado'), ('__eq__', <method-wrapper '__eq__' of Empleado object at 0x7f758826cc88>), ('__format__', <built-in method __format__ of Empleado object at 0x7f758826cc88>), ('__ge__', <method-wrapper '__ge__' of Empleado object at 0x7f758826cc88>), ('__getattribute__', <method-wrapper '__getattribute__' of Empleado object at 0x7f758826cc88>), ('__gt__', <method-wrapper '__gt__' of Empleado object at 0x7f758826cc88>), ('__hash__', <method-wrapper '__hash__' of Empleado object at 0x7f758826cc88>), ('__init__', <bound method Empleado.__init__ of <__main__.Empleado object at 0x7f758826cc88>>), ('__le__',

Incluso podemos obtener el código

In [37]:
import inspect

print(inspect.getsource(Empleado.getNombre))

    def getNombre(self):
        return self.nombre



## ¿Y para qué me vale esto?

La metaprogamación es especialmente útil cuando:

* Construyes herramientas para otros programadores (ej: un IDE)
* Quieres aprender cómo funciona realmente el lenguaje por dentro... y manipularlo
* ...

Es posible que estos argumentos no te parezcan suficientes. No hay problema. [Tim Peters](http://c2.com/cgi/wiki?TimPeters) dijo esto acerca de las metaclases (uno de los componentes fundamentales de la metaprogramación):

_[Metaclasses] are deeper magic than 99% of
users should ever worry about. If you
wonder whether you need them, you
don't (the people who actually need them
know with certainty that they need them,
and don't need an explanation about why)._

Así que, si no te ha surgido la necesidad de usar metaprogramación, no te preocupes ;-). Nosotros, al menos, vamos a ver un ejemplo de uso

## Caso de uso: depuración de código

¿Cómo depuramos nuestro código? Una primera aproximación (pobre) es... usando print

In [2]:
# Así es como depuran los muggles
def add(x, y):
    """
        Función que suma dos números
    """
    print('Dentro de la función add')
    return x + y

add(2, 2)

Dentro de la función add


4

Precioso. ¿Y si tenemos más funciones?

In [3]:
# La cosa se complica...
def add(x, y):
    """
        Función que suma dos números
    """
    print('add')
    return x + y

def sub(x, y):
    print('sub')
    return x - y

def mul(x, y):
    print('mul')
    return x * y

def div(x, y):
    print('div')
    return x / y

add(2, 2)
sub(8, 3)
mul(5,6)
div(16, 2)

add
sub
mul
div


8.0

Esto no es nada cómodo. Nos gustaría poder crear un *depurador* como elemento externo, y aplicarle ese elemento a todas las funciones que nos interese. Estamos hablando de los [decorators](https://wiki.python.org/moin/PythonDecorators)

<div class="alert alert-info">Los decorators no deberían ser algo ajeno para ti. Los vimos en la clase de orientación a objetos</div>

### Decoradores

Un decorador es una función que actúa como _wrapper_ de otra. El _wrapper_ actúa como la función original pero añade comportamiento adicional. En nuestro caso, lo que queremos es que añada información de debug

In [5]:
# Escribimos un decorador
def debug(func):
    
    # Ojo, que qualname vino en la 3.3: https://www.python.org/dev/peps/pep-3155/
    msg = func.__qualname__
    def wrapper(*args, **kwargs):
        print(msg)
        return func(*args, **kwargs)
    return wrapper

Hecho eso, podemos aplicar nuestro decorador así

In [6]:
# Aplicamos nuesto decorador a la función add
def add(x, y):
    """
        Función que suma dos números
    """
    return x + y

# Y llamamos a nuestra función add decorada
debug(add)(2,2)

add


4

Pero como esa manera de aplicar el decorador es bastante fea, utilizamos esta otra sintáxis

In [43]:
# Mejor usar esta sintáxis
@debug
def add(x, y):
    """
        Función que suma dos números
    """
    return x + y

add(2, 2)

add


4

Vale, pero ahora tenemos otro problema. ¿Qué pasa si yo quiero obtener ayuda acerca de la función *add*, para saber lo que hace?

In [45]:
# ¿Qué hace add?
help(add)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



Ups. Estamos obteniendo información sobre la función *wrapper*, que es quien se está haciendo cargo ahora de la función *add*. 

In [46]:
# Comprobación
add.__name__

'wrapper'

Para arreglar esto, el módulo *functools*, que vimos en el tema anterior, proporciona un decorador especial: [wraps](https://docs.python.org/3.5/library/functools.html#functools.wraps). Sirve, precisamente, para copiar la información de la función contenida al contenedor. Lo usamos así

In [47]:
from functools import wraps

def debug(func):
    msg = func.__qualname__
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(msg)
        return func(*args, **kwargs)
    return wrapper

Vamos ahora a redefinir nuestras funciones, aplicando el _wrapper_

In [52]:
@debug
def add(x, y):
    """
        Función que suma dos números
    """
    return x + y
@debug
def sub(x, y):
    return x - y
@debug
def mul(x, y):
    return x * y
@debug
def div(x, y):
    return x / y

add(2, 2)
sub(8, 3)
mul(5,6)
div(16, 2)

help(add)

add
sub
mul
div
Help on function add in module __main__:

add(x, y)
    Función que suma dos números



Por cierto, si quisiéramos *deshacer* el wrapper, aun podemos llamar a la función original:

In [51]:
# Accediendo al método sin decorar
original_add = add.__wrapped__
original_add(3, 4)

7

Con este cambio:

* Hemos accedido a la firma de una función (su nombre) para sacar información por pantalla (reflexión)
* Hemos aislado el código de debug en una función aparte, que puede ser aplicada a otras (separación de responsabilidades)
* Hemos copiado la información relevante de la función contenida a la función contenedora, con wraps

Eso hace feliz a Guido

<img src="https://www.soldierx.com/system/files/hdb/guido-van-rossum.jpg">

Una posible mejora sería utilizar el módulo de logger, que funciona de una manera muy similar al de Java

In [8]:
from functools import wraps
import logging

def debug(func):
    log = logging.getLogger(func.__module__)
    msg = func.__qualname__
    @wraps(func)
    def wrapper(*args, **kwargs):
        log.debug(msg)
        return func(*args, **kwargs)
    return wrapper

Otra adicional sería desactivar el mensaje de DEBUG usando una variable de entorno

In [10]:
from functools import wraps
import logging
import os

def debug(func):
    if 'DEBUG' not in os.environ:
        return func
    log = logging.getLogger(func.__module__)
    msg = func.__qualname__
    @wraps(func)
    def wrapper(*args, **kwargs):
        log.debug(msg)
        return func(*args, **kwargs)
    return wrapper

Imaginemos ahora que queremos añadir un prefijo a todos nuestros mensajes de depuración, y que ese prefijo sea dinámico. De otra forma: __queremos que nuestro decorador acepte parámetros__. Es decir, queremos hacer una llamada de este tipo (supongamos que queremos añadir un prefijo delante del mensaje decorado):

```python
debugargs(prefix="", add)(2, 2)```

Pero, por desgracia, un decorador solo acepta un parámetro: la función a decorar. ¿Cómo lo hacemos?

Debemos modificar ligeramente la llamada. Esto sí que se puede hacer:

```python
debugargs(prefix="")(add)(2, 2)```

¿No lo ves claro? Pongámoslo en dos pasos


```python
debug = debugargs(prefix="") # Esto es una llamada a una funcion que devuelve un decorador
debug(add)(2, 2) # Esta linea es igual que antes```

Vamos a traducirlo en palabras: __tenemos que crear una función que reciba un argumento (el prefijo). Esta función, a su vez, llamará al decorador. El decorador hará lo que tenga que hacer (mostrar un mensaje por pantalla) y luego llamará a la función original__. Son __3 niveles de profundidad__.

Vamos a escribirlo en código

In [4]:
import functools

def debugargs(prefix=''):
    '''
    Un decorador que acepta argumentos
    '''
    def debug(func):
        '''
        El decorador en si
        '''
        msg = prefix + func.__qualname__
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print(msg)
            return func(*args, **kwargs)
        return wrapper
    return debug

In [5]:
@debugargs(prefix="------")
def add(x, y):
    return x + y
@debugargs(prefix="------")
def sub(x, y):
    return x - y
@debugargs(prefix="------")
def mul(x, y):
    return x * y
@debugargs(prefix="------")
def div(x, y):
    return x / y


add(2, 2)
sub(8, 3)
mul(5,6)
div(16, 2)

------add
------sub
------mul
------div


8.0

Vale, eso está muy bien, pero ahora tengo dos wrappers: uno sin argumentos, y otro con argumentos. A mí lo que me gustaría es poder llamar al mismo, y decidir si quiero pasarle argumento o no.

Como ya conozco la programación funcional, puedo usar una de sus herramientas más poderosas en Python. La función [partial](https://docs.python.org/3/library/functools.html#functools.partial). Nos va a servir para ahorrarnos un nivel de profundidad, y para que nos explote el cerebro también.

In [15]:
import functools

def debug(func=None, prefix=''):
    '''
    Decorador que permite llamada tanto sin argumentos como con ellos
    '''
    if func is None:
        # Si la llamada se ha hecho sin método, es la llamada con atributos, de manera que devolvemos
        # una llamada al decorador, con un valor ya prefijado para prefix
        return functools.partial(debug, prefix=prefix)
    
    msg = prefix + func.__qualname__
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print(msg)
        return func(*args, **kwargs)
    return wrapper

Ahora puedo llamar a debug sin argumentos, o con un argumento opcional

In [16]:
@debug(prefix="++++++++")
def add(x, y):
    return x + y
@debug
def sub(x, y):
    return x - y
@debug(prefix="********")
def mul(x, y):
    return x * y
@debug
def div(x, y):
    return x / y


add(2, 2)
sub(8, 3)
mul(5,6)
div(16, 2)

++++++++add
sub
********mul
div


8.0

Eso también hace feliz a Guido...

<img src="https://www.soldierx.com/system/files/hdb/guido-van-rossum.jpg">

Vayamos un poco más lejos... ¿Qué pasa si quiero decorar todos los métodos de una clase? En vez de decorar método a método, podemos decorar la clase, de manera que apliquemos el decorador *debug* a todos sus métodos

In [22]:
# Decorando los métodos de una clase
def debugmethods(cls):
    for name, val in vars(cls).items():
        if callable(val):
            setattr(cls, name, debug(val)) # Aqui estamos aplicando el decorador debug anterior
    return cls

@debugmethods
class MiClase:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def foo(self):
        pass
    
    def bar(self):
        pass
    
    
c = MiClase(2, 3)
c.foo()
c.bar()


MiClase.__init__
MiClase.foo
MiClase.bar


También podríamos añadir un decorador a los atributos, cada vez que los obtenemos...

In [26]:
# Decorador para los atributos de una clase
def debugattr(cls):
    orig_getattribute = cls.__getattribute__

    def __getattribute__(self, name):
        print('Get:', name)
        return orig_getattribute(self, name)

    cls.__getattribute__ = __getattribute__

    return cls


# Decorando la clase con dos decoradores a la vez
@debugattr
@debugmethods
class Punto:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
p = Punto(2, 3)

print(p.x)
print(p.y)

Punto.__init__
Get: x
2
Get: y
3


Guido no puede ser más feliz...

<div class="row"><div class="col-md-6"><img src="https://www.soldierx.com/system/files/hdb/guido-van-rossum.jpg"></div><div class="col-md-6">
<img src="https://www.soldierx.com/system/files/hdb/guido-van-rossum.jpg"></div></div>

Ahora, cualquier clase puede ser decorada. Pero podemos hacerlo mejor. __DEBEMOS__ hacerlo mejor, para acercarnos al nirvana Pythoniano...

Estamos hablando, por supuesto, de las __metaclases__

### Metaclases

Todo en Python es un objeto. Eso incluye a las __clases__. Y al igual que una clase sirve para definir el comportamiento que va a tener un objeto, una *metaclase* sirve para definir el comportamiento que va a tener una clase.

Si no se especifica lo contrario, la metaclase asociada a toda clase creada en Python es *type*. Pero nada nos impide escribir nuestra propia metaclase que herede de *type*, e incluir el comportamiento que nos interese. 

Veamos un ejemplo

In [46]:
# Creando una metaclase
class debugmeta(type):
    def __new__(cls, clsname, bases, clsdict):
        clsobj = super().__new__(cls, clsname, bases, clsdict) # Clase creada
        clsobj = debugmethods(clsobj) # Añadimos un wrapper de clase
        return clsobj
    
# Usando la metaclase.
class MiClase(metaclass=debugmeta):
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def foo(self):
        pass
    
    def bar(self):
        pass
    
c = MiClase(2, 3)
c.foo()
c.bar()

MiClase.__init__
MiClase.foo
MiClase.bar


<div class="row"><div class="col-md-4"><img src="https://www.soldierx.com/system/files/hdb/guido-van-rossum.jpg"></div><div class="col-md-4">
<img src="https://www.soldierx.com/system/files/hdb/guido-van-rossum.jpg"></div><div class="col-md-4">
<img src="https://www.soldierx.com/system/files/hdb/guido-van-rossum.jpg"></div></div>

## Descriptores

En el tema 2, vimos que existía la posibilidad de convertir atributos de una clase en métodos, mediante el decorador *@property*. También era posible crear *setters* y *deleters*. Recuperamos uno de los ejemplos

In [54]:
# Aquí haremos algunas validaciones dentro de nuestros métodos get/set
class Person:
    def __init__(self, name, edad=18):
        self.__name = name
        self.edad = edad
        
    @property
    def name(self):
        return "{} es una bestia sexy".format(self.__name) if self.__name == "Jorge" else self.__name
    
    @name.setter
    def name(self, value):
        self.__name = value
        
    @property
    def edad(self):
        return self.__edad
    
    @edad.setter
    def edad(self, value):
        if value > 17:
            self.__edad = value
        else:
            print("No puedes entrar a la discoteca, eres menor de edad")
        
        
p = Person('Paco')
print(p.name)

p.edad = 15

p.name = 'Jorge'
print(p.name)

Paco
No puedes entrar a la discoteca, eres menor de edad
Jorge es una bestia sexy


La sintáxis es bastante fea, además de que es fácil equivocarse y acabar entrando en un bucle recursivo. La metaprogramación nos da una manera más elegante de obtener lo mismo, mediante el uso de descriptores. Son una manera de *empaquetar* las 3 operaciones básicas que se pueden realizar con un atributo: *get*, *set*, *delete*. 

Lo vemos en el ejemplo

In [59]:
# Creamos un descriptor
class Descriptor:
  # No nos hace falta get solo devolvería el valor
  def __init__(self, name=None):
    self.name = name
  def __set__(self, instance, value):
    instance.__dict__[self.name] = value

  # No dejamos borrar ningún campo
  def __delete__(self, instance):
    raise AttributeError("Can't delete")
    
#Y lo aplicamos a nuestra clase
class Persona:
    def __init__(self, name, edad=18):
        self.name = name
        self.edad = edad
        
    name = Descriptor('name')
    edad = Descriptor('edad')
    
# Ahora llamamos a nuestra clase
p = Persona('Paco')
p.edad = 21
p.name = 'Jorge'

print(p.edad)
print(p.name)

del(p.name)

21
Jorge


AttributeError: Can't delete

Ya somos los *dueños del punto*, porque controlamos las asignaciones y borrados de atributos de una clase. Ahora podemos hacer dos tipos de comprobaciones:

* Comprobación de tipos
* Comprobación de valores

### Comprobación de tipos

Por ejemplo, así evitamos que intenten meternos texto o en un campo numérico, o viceversa

In [72]:
# Clases para la comprobación de tipos
class Typed(Descriptor):
    ty = object
    def __set__(self, instance, value):
        if not isinstance(value, self.ty):
            raise TypeError('Expected {}'.format(self.ty))
        super().__set__(instance, value)
        
class Integer(Typed):
    ty = int
    
class String(Typed):
    ty = str

In [73]:
#Y lo aplicamos a nuestra clase
class Persona:
    def __init__(self, name, edad=18):
        self.name = name
        self.edad = edad
        
    name = String('name')
    edad = Integer('edad')
    
# Ahora llamamos a nuestra clase
p = Persona('Paco')
p.edad = 21
p.name = 'Jorge'

print(p.edad)
print(p.name)

# Solo permitimos un entero
p.edad = "100 años"

21
Jorge


TypeError: Expected <class 'int'>

La comprobación de tipos sería el nivel básico. Vamos a ver cómo haríamos una comprobación a otro nivel de abstracción. Por ejemplo, queremos evitar menores de edad

### Comprobación de valores

Añadamos la restricción de solo mayores de edad

In [69]:
class MayorDeEdad(Descriptor):
    def __set__(self, instance, value):
        if value < 18:
            raise ValueError('Expected >= 18')
        super().__set__(instance, value)
        
# Como hay herencia múltiple...
class IntegerMayorDeEdad(Integer, MayorDeEdad):
    pass

In [70]:
#Y lo aplicamos a nuestra clase
class Persona:
    def __init__(self, name, edad=18):
        self.name = name
        self.edad = edad
        
    name = String('name')
    edad = IntegerMayorDeEdad('edad')
    
# Ahora llamamos a nuestra clase
p = Persona('Paco')
p.edad = 21
p.name = 'Jorge'

print(p.edad)
print(p.name)

# Solo permitimos un entero y mayor de edad
p.edad = 12

21
Jorge


ValueError: Expected >= 18

<div class="row"><div class="col-md-6"><img src="https://www.soldierx.com/system/files/hdb/guido-van-rossum.jpg"></div><div class="col-md-6">
<img src="https://media.giphy.com/media/b9aScKLxdv0Y0/giphy.gif"></div></div>

<div class="alert alert-danger">__OJO__: Añadir demasiado _boilerplate_ a la creación de los ladrillos básicos de nuestra aplicación no es gratis. El rendimiento se puede ver seriamente perjudicado si metemos demasiadas comprobaciones, o potencialmente lentas</div>

---
_Las siguientes celdas contienen configuración del Notebook_

_Para visualizar y utlizar los enlaces a Twitter el notebook debe ejecutarse como [seguro](http://ipython.org/ipython-doc/dev/notebook/security.html)_

    File > Trusted Notebook

In [1]:
# Esta celda da el estilo al notebook
from IPython.core.display import HTML
css_file = '../static/styles/style.css'
HTML(open(css_file, "r").read())